## Install the needed packages

In [4]:
#Geocode Package

import sys
!{sys.executable} -m pip install geocoder
print('package installed')

package installed


In [5]:
# install geopy package

!conda install -c conda-forge geopy --yes
print('package installed')

Solving environment: done

# All requested packages already installed.

package installed


In [6]:
# install folium package

!conda install -c conda-forge folium=0.5.0 --yes
print('package installed')

Solving environment: done

# All requested packages already installed.

package installed


In [9]:
#import libraries

import pandas as pd  # library for data analsysis
import requests  # library to handle requests
import geocoder
import json  # library to handle JSON files
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import numpy as np  # library to handle data in a vectorized manner
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values
from sklearn.cluster import KMeans  # import k-means from clustering stage
import folium  # map rendering library

In [10]:
# Get neighborhood data: 'cod_distrito', 'distrito', 'cod_barrio', 'barrio' 

neighborhoods = pd.read_csv('BARRIOS.csv',sep=';')
neighborhoods = neighborhoods.rename(columns={'Codigo de distrito al que pertenece': 'Codigo de distrito'})
neighborhoods = neighborhoods.drop(columns={'Superficie (m2)', 'Perimetro (m)'})

neighborhoods

,Codigo de barrio,Codigo de distrito,Nombre de barrio
0,1,1,PALACIO
1,1,2,IMPERIAL
2,1,3,PACIFICO
3,1,4,RECOLETOS
4,1,5,EL VISO
...,...,...,...
126,7,20,CANILLEJAS
127,8,8,EL GOLOSO
128,8,15,ATALAYA
129,8,20,EL SALVADOR


In [11]:
boroughs = pd.read_csv('DISTRITOS.csv',sep=';')
boroughs = boroughs.drop(columns={'Superficie (m2)', 'Perimetro (m)','Numero de barrios'})
boroughs

,Codigo de distrito,Nombre de distrito
0,1,CENTRO
1,2,ARGANZUELA
2,3,RETIRO
3,4,SALAMANCA
4,5,CHAMARTIN
5,6,TETUAN
6,7,CHAMBERI
7,8,FUENCARRAL-EL PARDO
8,9,MONCLOA-ARAVACA
9,10,LATINA


In [12]:
boroughs_mad = pd.merge(boroughs, neighborhoods, how="left", on=["Codigo de distrito"])
boroughs_mad = boroughs_mad.drop(columns={'Codigo de barrio'})
boroughs_mad = boroughs_mad.rename(columns={'Nombre de distrito':'borough','Nombre de barrio':'neighborhood','Codigo de distrito':'borough code'})
boroughs_mad['Latitude'] = 0.0
boroughs_mad['Longitude'] = 0.0
boroughs_mad

,borough code,borough,neighborhood,Latitude,Longitude
0,1,CENTRO,PALACIO,0.0,0.0
1,1,CENTRO,EMBAJADORES,0.0,0.0
2,1,CENTRO,CORTES,0.0,0.0
3,1,CENTRO,JUSTICIA,0.0,0.0
4,1,CENTRO,UNIVERSIDAD,0.0,0.0
...,...,...,...,...,...
126,21,BARAJAS,ALAMEDA DE OSUNA,0.0,0.0
127,21,BARAJAS,AEROPUERTO,0.0,0.0
128,21,BARAJAS,CASCO H.BARAJAS,0.0,0.0
129,21,BARAJAS,TIMON,0.0,0.0


In [13]:
# call for arcgis to assess the coordinates for each Neighborhood
for i in range(0, len(boroughs_mad)):
    dir = boroughs_mad.iloc[i,2] + ', ' + boroughs_mad.iloc[i,1] + ', madrid, españa'  # "Neighborhood, Borough, 'madrid', 'españa'"
    g = geocoder.arcgis(dir)
    boroughs_mad.iloc[i,3] = g.latlng[0]
    boroughs_mad.iloc[i,4] = g.latlng[1]
    
boroughs_mad

,borough code,borough,neighborhood,Latitude,Longitude
0,1,CENTRO,PALACIO,40.40958,-3.88009
1,1,CENTRO,EMBAJADORES,40.39152,-3.69289
2,1,CENTRO,CORTES,40.41607,-3.69893
3,1,CENTRO,JUSTICIA,40.42479,-3.69308
4,1,CENTRO,UNIVERSIDAD,40.42565,-3.70726
...,...,...,...,...,...
126,21,BARAJAS,ALAMEDA DE OSUNA,40.45818,-3.58953
127,21,BARAJAS,AEROPUERTO,40.46148,-3.57666
128,21,BARAJAS,CASCO H.BARAJAS,40.36349,-5.14696
129,21,BARAJAS,TIMON,40.47366,-3.57777


In [14]:
# Foursquare credentials
CLIENT_ID = '5PDPRZ2A2ICKWCPF40WJYJGMXP24NH1QDS3EFI2WOX1K2HV1'
CLIENT_SECRET = 'HLPXY3G4VQDS4IIKVVZUJEDN5ZRKYL4GV1KG2I0WTMPB5YVJ'
VERSION = '20201110'

In [18]:
# define the venues dataframe
venues_columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'venue', 
                  'venue Latitude', 
                  'venue Longitude', 
                  'venue Category']

madrid_venues = pd.DataFrame(columns=venues_columns)

In [69]:
# function finding venues for a given location
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06377d81259'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'venue', 
                  'venue Latitude', 
                  'venue Longitude', 
                  'venue Category']
    
    return(nearby_venues)

In [70]:
#call the function
madrid_venues = getNearbyVenues(names=boroughs_mad['neighborhood'],
                                   latitudes=boroughs_mad['Latitude'],
                                   longitudes=boroughs_mad['Longitude'])

madrid_venues

PALACIO             
EMBAJADORES         
CORTES              
JUSTICIA            
UNIVERSIDAD         
SOL                 
IMPERIAL            
ACACIAS             
CHOPERA             
LEGAZPI             
DELICIAS            
PALOS DE MOGUER     
ATOCHA              
PACIFICO            
ADELFAS             
ESTRELLA            
IBIZA               
LOS JERONIMOS       
NI—O JESUS          
RECOLETOS           
GOYA                
FUENTE DEL BERRO    
GUINDALERA          
LISTA               
CASTELLANA          
EL VISO             
PROSPERIDAD         
CIUDAD JARDIN       
HISPANOAMERICA      
NUEVA ESPA—A        
CASTILLA            
BELLAS VISTAS       
CUATRO CAMINOS      
CASTILLEJOS         
ALMENARA            
VALDEACEDERAS       
BERRUGUETE          
GAZTAMBIDE          
ARAPILES            
TRAFALGAR           
ALMAGRO             
RIOS ROSAS          
VALLEHERMOSO        
EL PARDO            
FUENTELARREINA      
PE—A GRANDE         
EL PILAR            
LA PAZ       

,neighborhood,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
0,PALACIO,40.40958,-3.88009,BeOne,40.404435,-3.885560,Gym / Fitness Center
1,PALACIO,40.40958,-3.88009,Skatepark de Boadilla,40.410489,-3.888021,Skate Park
2,PALACIO,40.40958,-3.88009,Fitness19,40.409288,-3.880009,Gym / Fitness Center
3,PALACIO,40.40958,-3.88009,Parque Sofía de Grecia II,40.409460,-3.882442,Plaza
4,PALACIO,40.40958,-3.88009,Parque Santillana del Mar,40.407508,-3.882004,Park
...,...,...,...,...,...,...,...
2930,CORRALEJOS,40.46540,-3.61164,Recinto Para Perros Parque JCI,40.462782,-3.608200,Dog Run
2931,CORRALEJOS,40.46540,-3.61164,Piscina de Novotel,40.462584,-3.614951,Pool
2932,CORRALEJOS,40.46540,-3.61164,Fitness Pullman Hotel,40.462311,-3.614936,Gym / Fitness Center
2933,CORRALEJOS,40.46540,-3.61164,Lago del Parque Juan Carlos I,40.460311,-3.609649,Lake


In [71]:
madrid_venues.groupby('neighborhood').count()

,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
neighborhood,,,,,,
ABRANTES,16,16,16,16,16,16
ACACIAS,56,56,56,56,56,56
ADELFAS,24,24,24,24,24,24
AEROPUERTO,6,6,6,6,6,6
ALAMEDA DE OSUNA,14,14,14,14,14,14
...,...,...,...,...,...,...
VENTAS,8,8,8,8,8,8
VILLAVERDE ALTO C.H.,4,4,4,4,4,4
VINATEROS,12,12,12,12,12,12


In [72]:
madrid_venues.head()

,neighborhood,neighborhood Latitude,neighborhood Longitude,venue,venue Latitude,venue Longitude,venue Category
0,PALACIO,40.40958,-3.88009,BeOne,40.404435,-3.885560,Gym / Fitness Center
1,PALACIO,40.40958,-3.88009,Skatepark de Boadilla,40.410489,-3.888021,Skate Park
2,PALACIO,40.40958,-3.88009,Fitness19,40.409288,-3.880009,Gym / Fitness Center
3,PALACIO,40.40958,-3.88009,Parque Sofía de Grecia II,40.409460,-3.882442,Plaza
4,PALACIO,40.40958,-3.88009,Parque Santillana del Mar,40.407508,-3.882004,Park


In [73]:
madrid_onehot = pd.get_dummies(madrid_venues[['venue Category']], prefix="", prefix_sep="")
madrid_onehot

,Athletics & Sports,Basketball Court,Botanical Garden,Boxing Gym,Campground,Cycle Studio,Dog Run,Fountain,Garden,Golf Course,...,Soccer Field,Sports Club,Stables,Summer Camp,Tennis Court,Track,Trail,Tree,Volleyball Court,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2931,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
madrid_onehot.insert(loc=0, column='neighborhood', value=madrid_venues['neighborhood'] )
madrid_onehot

,neighborhood,Athletics & Sports,Basketball Court,Botanical Garden,Boxing Gym,Campground,Cycle Studio,Dog Run,Fountain,Garden,...,Soccer Field,Sports Club,Stables,Summer Camp,Tennis Court,Track,Trail,Tree,Volleyball Court,Yoga Studio
0,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PALACIO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,CORRALEJOS,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2931,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2932,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2933,CORRALEJOS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
madrid_grouped = madrid_onehot.groupby('neighborhood').mean().reset_index()
madrid_grouped

,neighborhood,Athletics & Sports,Basketball Court,Botanical Garden,Boxing Gym,Campground,Cycle Studio,Dog Run,Fountain,Garden,...,Soccer Field,Sports Club,Stables,Summer Camp,Tennis Court,Track,Trail,Tree,Volleyball Court,Yoga Studio
0,ABRANTES,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,ACACIAS,0.017857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053571,...,0.017857,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.053571
2,ADELFAS,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.041667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,AEROPUERTO,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.166667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,ALAMEDA DE OSUNA,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,...,0.000000,0.0,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,VENTAS,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
125,VILLAVERDE ALTO C.H.,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
126,VINATEROS,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,...,0.250000,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.000000
127,VISTA ALEGRE,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [77]:
#function returning categories for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
# get top ten common venues for each Neighborhood
num_top_venues = 14

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = madrid_grouped['neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
0,ABRANTES,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Playground,Roof Deck,Mountain,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Gun Range
1,ACACIAS,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports
2,ADELFAS,Gym,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Harbor / Marina,Martial Arts School,Skate Park,Soccer Field,Gymnastics Gym,Gym Pool,Lake,Basketball Court,Indoor Play Area
3,AEROPUERTO,Sculpture Garden,Playground,Plaza,Soccer Field,Garden,Gym,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gun Range
4,ALAMEDA DE OSUNA,Park,Plaza,Gym,Gym / Fitness Center,Tennis Court,Garden,Scenic Lookout,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gun Range
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,VENTAS,Park,Soccer Field,Athletics & Sports,Plaza,Gym / Fitness Center,Gun Range,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Golf Course,Mountain
125,VILLAVERDE ALTO C.H.,Skate Park,Park,Plaza,Garden,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Gun Range,Mountain
126,VINATEROS,Soccer Field,Athletics & Sports,Park,Trail,Plaza,Pool,Garden,Martial Arts School,Gym,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center
127,VISTA ALEGRE,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Playground,Roof Deck,Mountain,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Gun Range


In [79]:
# Clustering for Neighborhood

# set number of clusters
kclusters = 5

madrid_grouped_clustering = madrid_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 3, 3, 0, 3, 0, 1, 0], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = boroughs_mad


In [81]:
# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')

madrid_merged.head()

,borough code,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
0,1,CENTRO,PALACIO,40.40958,-3.88009,0.0,Gym / Fitness Center,Plaza,Athletics & Sports,Park,Pool,Skate Park,Cycle Studio,Dog Run,Lake,Basketball Court,Indoor Play Area,Harbor / Marina,Botanical Garden,Boxing Gym
1,1,CENTRO,EMBAJADORES,40.39152,-3.69289,3.0,Park,Plaza,Gym,Playground,Gym / Fitness Center,Soccer Field,Yoga Studio,Garden,Pool,Skating Rink,Skate Park,Fountain,Lake,Golf Course
2,1,CENTRO,CORTES,40.41607,-3.69893,0.0,Plaza,Gym,Gym / Fitness Center,Park,Athletics & Sports,Garden,Roof Deck,Fountain,Yoga Studio,Sculpture Garden,Pedestrian Plaza,Lake,Playground,Tree
3,1,CENTRO,JUSTICIA,40.42479,-3.69308,0.0,Plaza,Park,Yoga Studio,Gym,Gym / Fitness Center,Athletics & Sports,Gymnastics Gym,Fountain,Playground,Garden,Gym Pool,Outdoors & Recreation,Sports Club,Trail
4,1,CENTRO,UNIVERSIDAD,40.42565,-3.70726,0.0,Plaza,Gym,Gym / Fitness Center,Park,Athletics & Sports,Garden,Gym Pool,Gymnastics Gym,Outdoors & Recreation,Roof Deck,Pedestrian Plaza,Playground,Yoga Studio,Sculpture Garden


In [82]:
neighborhoods_venues_sorted

,Cluster Labels,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
0,0,ABRANTES,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Playground,Roof Deck,Mountain,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Gun Range
1,0,ACACIAS,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports
2,3,ADELFAS,Gym,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Harbor / Marina,Martial Arts School,Skate Park,Soccer Field,Gymnastics Gym,Gym Pool,Lake,Basketball Court,Indoor Play Area
3,3,AEROPUERTO,Sculpture Garden,Playground,Plaza,Soccer Field,Garden,Gym,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gun Range
4,3,ALAMEDA DE OSUNA,Park,Plaza,Gym,Gym / Fitness Center,Tennis Court,Garden,Scenic Lookout,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gun Range
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1,VENTAS,Park,Soccer Field,Athletics & Sports,Plaza,Gym / Fitness Center,Gun Range,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Golf Course,Mountain
125,0,VILLAVERDE ALTO C.H.,Skate Park,Park,Plaza,Garden,Yoga Studio,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Gun Range,Mountain
126,1,VINATEROS,Soccer Field,Athletics & Sports,Park,Trail,Plaza,Pool,Garden,Martial Arts School,Gym,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center
127,0,VISTA ALEGRE,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Playground,Roof Deck,Mountain,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym,Gun Range


In [83]:
#searching location for 'madrid, españa'

address = 'madrid, españa'

geolocator = Nominatim(user_agent="mad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


In [84]:
# Map with clusters for 'Madrid, España'
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

## Examine Clusters

### Cluster 1

In [85]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
0,CENTRO,0.0,Gym / Fitness Center,Plaza,Athletics & Sports,Park,Pool,Skate Park,Cycle Studio,Dog Run,Lake,Basketball Court,Indoor Play Area,Harbor / Marina,Botanical Garden,Boxing Gym
2,CENTRO,0.0,Plaza,Gym,Gym / Fitness Center,Park,Athletics & Sports,Garden,Roof Deck,Fountain,Yoga Studio,Sculpture Garden,Pedestrian Plaza,Lake,Playground,Tree
3,CENTRO,0.0,Plaza,Park,Yoga Studio,Gym,Gym / Fitness Center,Athletics & Sports,Gymnastics Gym,Fountain,Playground,Garden,Gym Pool,Outdoors & Recreation,Sports Club,Trail
4,CENTRO,0.0,Plaza,Gym,Gym / Fitness Center,Park,Athletics & Sports,Garden,Gym Pool,Gymnastics Gym,Outdoors & Recreation,Roof Deck,Pedestrian Plaza,Playground,Yoga Studio,Sculpture Garden
5,CENTRO,0.0,Plaza,Gym,Gym / Fitness Center,Garden,Yoga Studio,Athletics & Sports,Roof Deck,Outdoors & Recreation,Park,Sculpture Garden,Fountain,Palace,Gymnastics Gym,Trail
6,ARGANZUELA,0.0,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports
7,ARGANZUELA,0.0,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports
8,ARGANZUELA,0.0,Park,Plaza,Gym,Playground,Pool,Yoga Studio,Gym / Fitness Center,Soccer Field,Skate Park,Garden,Pedestrian Plaza,Athletics & Sports,Skating Rink,Tennis Court
9,ARGANZUELA,0.0,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports
10,ARGANZUELA,0.0,Plaza,Park,Gym,Gym / Fitness Center,Yoga Studio,Garden,Playground,Pool,Gym Pool,Gymnastics Gym,Lake,Outdoors & Recreation,Pedestrian Plaza,Athletics & Sports


### Cluster 2

In [88]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
16,RETIRO,1.0,Park,Plaza,Garden,Fountain,Gym,Gym / Fitness Center,Lake,Yoga Studio,Harbor / Marina,Athletics & Sports,Playground,Sports Club,Scenic Lookout,Outdoors & Recreation
60,LATINA,1.0,Park,Athletics & Sports,Plaza,Gym,Playground,Soccer Field,Gym / Fitness Center,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gun Range,Mountain
61,LATINA,1.0,Park,Athletics & Sports,Plaza,Gym,Playground,Soccer Field,Gym / Fitness Center,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gun Range,Mountain
63,LATINA,1.0,Park,Playground,Soccer Field,Skydiving Drop Zone,Yoga Studio,Gun Range,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Golf Course
72,USERA,1.0,Park,Athletics & Sports,Plaza,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gun Range,Outdoors & Recreation,Golf Course
73,USERA,1.0,Park,Plaza,Soccer Field,Yoga Studio,Gun Range,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Golf Course,Mountain
74,USERA,1.0,Park,Soccer Field,Athletics & Sports,Scenic Lookout,Trail,Tennis Court,Plaza,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Gun Range
79,PUENTE DE VALLECAS,1.0,Park,Trail,Yoga Studio,Gun Range,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Golf Course,Outdoors & Recreation
85,MORATALAZ,1.0,Soccer Field,Athletics & Sports,Park,Trail,Plaza,Pool,Garden,Martial Arts School,Gym,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center
86,MORATALAZ,1.0,Soccer Field,Athletics & Sports,Park,Trail,Plaza,Pool,Garden,Martial Arts School,Gym,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center


### Cluster 3

In [89]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
43,FUENCARRAL-EL PARDO,2.0,Scenic Lookout,Yoga Studio,Outdoors & Recreation,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Gun Range,Golf Course,Garden
45,FUENCARRAL-EL PARDO,2.0,Scenic Lookout,Yoga Studio,Outdoors & Recreation,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Gym,Gun Range,Golf Course,Garden


### Cluster 4

In [90]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
1,CENTRO,3.0,Park,Plaza,Gym,Playground,Gym / Fitness Center,Soccer Field,Yoga Studio,Garden,Pool,Skating Rink,Skate Park,Fountain,Lake,Golf Course
13,RETIRO,3.0,Gym,Plaza,Park,Athletics & Sports,Martial Arts School,Harbor / Marina,Gym / Fitness Center,Playground,Gymnastics Gym,Skate Park,Soccer Field,Dog Run,Garden,Golf Course
14,RETIRO,3.0,Gym,Plaza,Gym / Fitness Center,Athletics & Sports,Park,Harbor / Marina,Martial Arts School,Skate Park,Soccer Field,Gymnastics Gym,Gym Pool,Lake,Basketball Court,Indoor Play Area
15,RETIRO,3.0,Park,Gym,Plaza,Athletics & Sports,Sports Club,Basketball Court,Soccer Field,Pool,Martial Arts School,Gym / Fitness Center,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool
18,RETIRO,3.0,Park,Plaza,Gym,Garden,Athletics & Sports,Sports Club,Harbor / Marina,Dog Run,Pool,Soccer Field,Fountain,Outdoors & Recreation,Martial Arts School,Basketball Court
19,SALAMANCA,3.0,Gym / Fitness Center,Gym,Plaza,Sculpture Garden,Park,Athletics & Sports,Gymnastics Gym,Playground,Boxing Gym,Garden,Volleyball Court,Pedestrian Plaza,Yoga Studio,Sports Club
20,SALAMANCA,3.0,Gym / Fitness Center,Gym,Plaza,Sculpture Garden,Park,Athletics & Sports,Gymnastics Gym,Playground,Boxing Gym,Garden,Volleyball Court,Pedestrian Plaza,Yoga Studio,Sports Club
21,SALAMANCA,3.0,Gym / Fitness Center,Park,Gym,Plaza,Athletics & Sports,Sculpture Garden,Gymnastics Gym,Garden,Outdoors & Recreation,Pool,Yoga Studio,Soccer Field,Skating Rink,Fountain
22,SALAMANCA,3.0,Gym / Fitness Center,Plaza,Gym,Park,Gymnastics Gym,Yoga Studio,Pool,Basketball Court,Garden,Volleyball Court,Athletics & Sports,Soccer Field,Skating Rink,Fountain
23,SALAMANCA,3.0,Gym / Fitness Center,Gym,Plaza,Sculpture Garden,Park,Athletics & Sports,Gymnastics Gym,Playground,Boxing Gym,Garden,Volleyball Court,Pedestrian Plaza,Yoga Studio,Sports Club


### Cluster 5

In [91]:
madrid_merged.loc[madrid_merged['Cluster Labels'] == 4, madrid_merged.columns[[1] + list(range(5, madrid_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue
44,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
46,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
47,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
48,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
49,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
50,FUENCARRAL-EL PARDO,4.0,Park,Mountain,Dog Run,Gym,Martial Arts School,Lake,Indoor Play Area,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Yoga Studio,Outdoors & Recreation,Golf Course
